# 1. Getting the Data

First we will scrape policies from the gov.ie website.

In your command line, ``cd`` into this repository.

``cd`` into the ``policy_scraping`` task directory, then ``cd`` again into the ``policy_scraping`` scrapy environment.

In [1]:
import os
cwd = os.getcwd() # should be base directory of repository
os.chdir(cwd+"/policy_scraping/policy_scraping")

Run ``scrapy crawl goviefor -O ../outputs/goviefor.json`` (or you can change the -O argument to whatever you would prefer the output file information to be).

This command will generate a json containing the metadata about all the policies as well as download all files to the same outputs directory under ``forestry/full``.

In [ ]:
!! scrapy crawl goviefor -O ../outputs/goviefor.json

['2025-03-02 14:41:45 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: policy_scraping)',
 '2025-03-02 14:41:45 [scrapy.utils.log] INFO: Versions: lxml 5.1.0.0, libxml2 2.10.3, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.10.0, Python 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.2, Platform Windows-10-10.0.26100-SP0',
 '2025-03-02 14:41:45 [scrapy.crawler] INFO: Overridden settings:',
 "{'BOT_NAME': 'policy_scraping',",
 " 'FEED_EXPORT_ENCODING': 'utf-8',",
 " 'NEWSPIDER_MODULE': 'policy_scraping.spiders',",
 " 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',",
 " 'ROBOTSTXT_OBEY': True,",
 " 'SPIDER_MODULES': ['policy_scraping.spiders'],",
 " 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor',",
 " 'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) '",
 "               'Gecko/20100101 Firefox/66.0'}",
 '2025-03-02 14:41:4

Next we will consolidate the metadata and text of the policy PDFs into one dictionary.

In [3]:
os.chdir(cwd) # back to base directory
from populate_corpora.pdfs_to_jsons import scrp_itm_to_fulltxt
import json
FILE_DIR= cwd+"/policy_scraping/policy_scraping/outputs" # or whatever output directory you gave the scraper for its output json

with open(cwd+"/policy_scraping/outputs/goviefor.json","r", encoding="utf-8") as f:
    metad = json.load(f)

pdf_dict = scrp_itm_to_fulltxt(metad, FILE_DIR+"/forestry/full")

  0%|          | 0/284 [00:00<?, ?it/s]

Could not read bd0e2215bffb644e8c4b3a7df2b26f3d4adabbe3 due to [WinError 2] The system cannot find the file specified: 'c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\bd0e2215bffb644e8c4b3a7df2b26f3d4adabbe3.pdf'


 10%|█         | 29/284 [00:48<11:49,  2.78s/it]

Could not read a57ebf5bbe57725807dab1e81a537b32ed163d2f due to [WinError 2] The system cannot find the file specified: 'c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\a57ebf5bbe57725807dab1e81a537b32ed163d2f.pdf'


 11%|█         | 31/284 [00:51<09:53,  2.35s/it]

...

Could not read bb424bf6b9db093a0cc8f73851a9a867bfb3f157 due to [WinError 2] The system cannot find the file specified: 'c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\bb424bf6b9db093a0cc8f73851a9a867bfb3f157.pdf'


 96%|█████████▌| 272/284 [04:03<00:39,  3.26s/it]

Could not read 4fb4f481f382f3a41cd7a2d10adaa8ac51808c77 due to [WinError 2] The system cannot find the file specified: 'c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\4fb4f481f382f3a41cd7a2d10adaa8ac51808c77.pdf'
Could not read d59e36515abb69d36be5a002bc98e8f2c3f7c4d4 due to [WinError 2] The system cannot find the file specified: 'c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\d59e36515abb69d36be5a002bc98e8f2c3f7c4d4.pdf'
Could not read a56cda5ddd99f870187b666e85c48f4e5b4bc119 due to [WinError 2] The system cannot find the file specified: 'c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\a56cda5ddd99f870187b666e85c48f4e5b4bc119.pdf'
Could not read 4ac9ddedcbd4ef847c47a0f2804de3cdf0d16dca due to [WinError 2] The system cannot find the file specified: 'c:\\Users\\allie\\Documents\\GitHub\\policy-classifi

100%|██████████| 284/284 [05:37<00:00,  1.19s/it]

Removed hash bd0e2215bffb644e8c4b3a7df2b26f3d4adabbe3 from dictionary
Removed hash a57ebf5bbe57725807dab1e81a537b32ed163d2f from dictionary
Removed hash 74ec9122f540da0cd334f6260ade67574fed5819 from dictionary
Removed hash 3298d4d9318d1dba58fca6a03fb4f084e5ee23a7 from dictionary
Removed hash a6ffd9f252c4b973a5f247b807ccb9ae06ea592c from dictionary
Removed hash 165434081592a52fdcbfbd25b289167f18510ead from dictionary
Removed hash 6d240c5d8ae1c6d9c97aa7573604106710cd22d3 from dictionary
Removed hash 8165d5880a6d5c2061958953caaa9858d9cbe6f5 from dictionary
Removed hash c7e10054e8036b71970e4773504fb0c91e59b7a0 from dictionary
Removed hash 322b959f91c817def6d7735dea4f615a7a11cac5 from dictionary
Removed hash bf00b2188c455415e77bcff46d403137d89c18aa from dictionary
Removed hash cde487733ecd498d0beb51689dc5a7b332863000 from dictionary
Removed hash a0907bbf1d4c6d457c04855ecd794fc0f8e71982 from dictionary
Removed hash 1aa6b403beac7649b27e104695ea12d688437009 from dictionary
Removed hash c665c1e

If you have your own collection of pdfs to process and don't have a metadata file, you can use this next function on just the file directory.

In [ ]:
from populate_corpora.pdfs_to_jsons import pdfs_to_txt_dct
pdf_dict = pdfs_to_txt_dct(FILE_DIR+"/forestry/full") # or whatever your policy directory is

 14%|█▍        | 21/149 [00:24<03:30,  1.64s/it]unknown widths : 
[0, IndirectObject(1289, 0, 2848620776528), 7, 8, 661, 9, IndirectObject(1291, 0, 2848620776528), 11, 12, 575, 13, IndirectObject(1306, 0, 2848620776528), 17, 18, 296, 19, IndirectObject(1307, 0, 2848620776528), 25, 26, 770, 27, 28, 808, 29, IndirectObject(1310, 0, 2848620776528), 34, 35, 548, 36, IndirectObject(1308, 0, 2848620776528), 38, 39, 742, 40, IndirectObject(1318, 0, 2848620776528), 45, 46, 507, 47, IndirectObject(1305, 0, 2848620776528), 55, 56, 567, 57, IndirectObject(1313, 0, 2848620776528), 59, 60, 534, 61, IndirectObject(1300, 0, 2848620776528), 67, 75, 600, 76, 84, 924, 85, IndirectObject(1301, 0, 2848620776528), 95, 97, 564, 98, 99, 254, 100, IndirectObject(1312, 0, 2848620776528), 104, 105, 534, 106, IndirectObject(1311, 0, 2848620776528), 109, 110, 564, 111, 112, 574, 113, IndirectObject(1296, 0, 2848620776528), 114, 115, 568, 116, IndirectObject(1317, 0, 2848620776528), 118, 119, 439, 120, IndirectObj

['Could not read c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\33120105aa142c885afe85a8b1fee7dfc4c97b32.pdf due to PyCryptodome is required for AES algorithm', 'Could not read c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\5d043c94be7ac309e00175ed1228e4cf31b59334.pdf due to PyCryptodome is required for AES algorithm', 'Could not read c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\7ba4749e076cd2d74d5928777773acddcc50a247.pdf due to PyCryptodome is required for AES algorithm', 'Could not read c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\bd0e2215bffb644e8c4b3a7df2b26f3d4adabbe3.xlsx due to EOF marker not found', 'Could not read c:\\Users\\allie\\Documents\\GitHub\\policy-classifier/policy_scraping/policy_scraping/outputs/forestry/full\\cc

For the purposes of this project, we only want the texts of the PDFs in cleaned sentences anyways. So we'll go ahead and extract/clean those sentences, then load them into the dictionary format that doccano (labeling platform) uses. Finally, if we want, we can use a simple keyword search to prelabel some of the sentences with a "incentive class mention" label.

In [30]:
import nltk
from populate_corpora.json_cleaning import get_clean_text_sents, format_sents_for_doccano, prelabeling
EN_TOKENIZER = nltk.data.load("tokenizers/punkt/english.pickle") # need tokenizer for our text cleaning
clean_sents= get_clean_text_sents(pdf_dict, EN_TOKENIZER)
doccano_dict = format_sents_for_doccano(clean_sents)
prelab_doccano_dict = prelabeling(doccano_dict)

100%|██████████| 134/134 [00:07<00:00, 17.25it/s]


Number of error files: 0


Now we can download this dictionary as a json to import into our doccano instance for labeling.

In [32]:
with open(FILE_DIR+f"/ready_to_label.json", 'w', encoding="utf-8") as outfile:
    json.dump(prelab_doccano_dict, outfile, ensure_ascii=False, indent=4)

# 2. Labeling the Data

We will now load the labeled dataset 